In [21]:
import duckdb
import pandas as pd

%load_ext sql
# Return Pandas DataFrames instead of regular result sets
%config SqlMagic.autopandas = True
# Suppress feedback messages
%config SqlMagic.feedback = False
# Show connection string after execution
%config SqlMagic.displaycon = False
# Connect to DuckDB in-memory database
conn = duckdb.connect('database.duckdb', read_only=True)

%sql conn --alias duckdb


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [22]:
conn.sql("""SELECT table_name, column_count, estimated_size FROM duckdb_tables ORDER by estimated_size DESC""").df()

,table_name,column_count,estimated_size
0,WDIfootnote,4,838817
1,WDICSV,69,401394
2,WDIcountryseries,3,8248
3,WDISeries,20,1509
4,WDICountry,31,265
5,WDIseriestime,3,148


In [23]:
table = "WDIseriestime"

In [24]:
df = conn.sql(f"SELECT * FROM {table}").df()
df.head()

,SeriesCode,Year,DESCRIPTION
0,SH.MED.PHYS.ZS,YR1990,Averages for regions/groups are based on count...
1,SP.UWT.TFRT,YR1990,Averages for regions/groups are based on count...
2,SP.REG.DTHS.ZS,YR1990,Averages for regions/groups are based on count...
3,SH.STA.ARIC.ZS,YR1990,Averages for regions/groups are based on count...
4,SP.DYN.WFRT,YR1990,Averages for regions/groups are based on count...


In [25]:
df.shape

(148, 3)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   SeriesCode   148 non-null    object
 1   Year         148 non-null    object
 2   DESCRIPTION  148 non-null    object
dtypes: object(3)
memory usage: 3.6+ KB


In [27]:
df.describe(include=object)

,SeriesCode,Year,DESCRIPTION
count,148,148,148
unique,37,18,13
top,IC.FRM.FEMO.ZS,YR2000,The sample was drawn from the manufacturing se...
freq,6,20,66


In [28]:
join = conn.sql(f"""SELECT * FROM WDIfootnote LEFT JOIN WDICSV ON WDIfootnote.CountryCode = WDICSV."Country Code" AND WDIfootnote.SeriesCode = WDICSV."Indicator Code" """).df()

In [29]:
len(join)

838817

In [30]:
join.head()

,CountryCode,SeriesCode,Year,DESCRIPTION,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,ESP,SL.EMP.WORK.FE.ZS,YR2020,Adjusted,Spain,ESP,"Wage and salaried workers, female (% of female...",SL.EMP.WORK.FE.ZS,NaN,NaN,...,87.080931,87.526877,87.534405,87.860606,88.017806,87.842282,87.922180,87.788735,88.286693,NaN
1,ESP,SL.EMP.WORK.MA.ZS,YR2021,Adjusted,Spain,ESP,"Wage and salaried workers, male (% of male emp...",SL.EMP.WORK.MA.ZS,NaN,NaN,...,78.919880,79.875720,79.827050,80.634902,81.142731,80.578465,81.248783,80.936719,81.882487,NaN
2,ESP,SL.EMP.WORK.ZS,YR2022,Adjusted,Spain,ESP,"Wage and salaried workers, total (% of total e...",SL.EMP.WORK.ZS,NaN,NaN,...,82.596575,83.324656,83.306828,83.899466,84.255816,83.865102,84.298487,84.066327,84.830662,NaN
3,LKA,SE.PRM.TENR,YR2007,UIS estimate,Sri Lanka,LKA,"Adjusted net enrollment rate, primary (% of pr...",SE.PRM.TENR,NaN,NaN,...,99.062710,99.060070,99.284540,99.273480,NaN,NaN,NaN,NaN,NaN,NaN
4,LKA,SE.PRM.TENR.FE,YR2007,UIS estimate,Sri Lanka,LKA,"Adjusted net enrollment rate, primary, female ...",SE.PRM.TENR.FE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
